# Python + Pytorch Starter Notebook

Notes from my adventure setting up a Jupyter Notebook for Pytorch on my mac.

## Does python work?

Select the cell below and hit Ctrl + Enter to execute.  I set up conda and had to configure the kernel.  ctrl-Shift-P Create Jupyter.

In [1]:
1+1

2

## Is Pytorch installed and available in this kernel?

Run this next:

In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1837, 0.8384, 0.7957],
        [0.3421, 0.3699, 0.4904],
        [0.4152, 0.1468, 0.4164],
        [0.0644, 0.8416, 0.6187],
        [0.2257, 0.6681, 0.1577]])


If you see the tensor output above, it's working!  

## Using Pytorch

Let's get some data, but first more imports.  

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Now we have some data to use for training.  Pass the dataset to a dataloader.  Dataloaders give you an iterable and handles things like shuffling/randomization for you.  

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating a Model

Let's create the model we will use.  The model is an object that represents the architecture of the network.  How many layers?  What kind of layers?  How do we move forward?

After we create an instance of the model, we send it `to` the device it will be run on (CUDA if you've got a GPU capable, my macbook is on mps, nad worst case you'll run on CPU.  Performance gets worse as you move down the list but it still works.)

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the model's Parameters

The model needs to know how to calculate loss and an optimizer.  I need to understand what these do better, but I know you're basically selecting from some math functions based on how you think they'll perform with the rest of your model.  Experimentation to find what works best with your data.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## Training

In this loop it trains itself and updates it's weights based on how it does.  We'll actually run it soon, when we write the training loop.

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

# Check our progress

The other function we need to implement our training loop is a test function.  There's a bunch I need to read more about in here.

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## The Training loop!

Here goes:

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310679  [   64/60000]
loss: 2.289166  [ 6464/60000]
loss: 2.268306  [12864/60000]
loss: 2.257291  [19264/60000]
loss: 2.237663  [25664/60000]
loss: 2.214821  [32064/60000]
loss: 2.225413  [38464/60000]
loss: 2.189602  [44864/60000]
loss: 2.183145  [51264/60000]
loss: 2.153429  [57664/60000]
Test Error: 
 Accuracy: 49.6%, Avg loss: 2.139284 

Epoch 2
-------------------------------
loss: 2.157562  [   64/60000]
loss: 2.137975  [ 6464/60000]
loss: 2.071871  [12864/60000]
loss: 2.090625  [19264/60000]
loss: 2.034302  [25664/60000]
loss: 1.976987  [32064/60000]
loss: 2.010672  [38464/60000]
loss: 1.926161  [44864/60000]
loss: 1.925032  [51264/60000]
loss: 1.863808  [57664/60000]
Test Error: 
 Accuracy: 59.0%, Avg loss: 1.846233 

Epoch 3
-------------------------------
loss: 1.882257  [   64/60000]
loss: 1.842091  [ 6464/60000]
loss: 1.715456  [12864/60000]
loss: 1.772566  [19264/60000]
loss: 1.659845  [25664/60000]
loss: 1.613323  [32064/600

## Wicked cool - lets save it

We've got a model.  It's not great - plenty of room to learn and improve the existing code.  Let's save the model:

In [11]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


It saved the file next to this notebook file, it's in the explorer panel.

## Loading the model

We can load an existing, saved model from disk like so

In [12]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [13]:
# Next lets use it - and disable backpropagation.

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
